# Make geojson for pulling MODIS data
- Read the site metadata generated in the previous step
- Calculate a 4km x 4km bounding box in WGS84
- Ensure time format is compatible with colab script

In [1]:
import os
import zipfile
import re
import pandas as pd
from collections import defaultdict
from shapely.geometry import Polygon, mapping
from pyproj import Proj, transform, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import geopandas as gpd

In [7]:
meta = pd.read_csv('processed_site_meta.csv')
meta

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,TIME_INFO
0,AR-TF1,-54.973300,-66.733500,40.0,"{'ameriflux': ['2016_01_01', '2018_12_31']}"
1,BR-CST,-7.968200,-38.384200,468.0,"{'ameriflux': ['2014_01_01', '2015_12_31']}"
2,BR-Npw,-16.498000,-56.412000,120.0,"{'ameriflux': ['2013_01_01', '2017_12_31']}"
3,BR-Sa3,-3.018000,-54.971400,100.0,"{'fluxnet': ['2000_01_01', '2004_12_31']}"
4,CA-ARB,52.695000,-83.945200,90.0,"{'ameriflux': ['2011_01_01', '2015_12_31']}"
...,...,...,...,...,...
381,CD-Ygb,0.814444,24.502472,466.0,"{'icos-2023': ['2020_01_01', '2022_12_31']}"
382,IT-Niv,45.490910,7.139430,2750.0,"{'icos-2023': ['2019_01_01', '2022_12_31']}"
383,FR-Mej,48.118400,-1.796350,40.0,"{'icos-2023': ['2019_01_01', '2022_12_31']}"
384,DK-Vng,56.037476,9.160710,67.7,"{'icos-2023': ['2020_01_01', '2022_12_31']}"


In [8]:
def get_4km_polygon_wkt(lat, lon):
    utm_crs_info = query_utm_crs_info(
        area_of_interest=AreaOfInterest(west_lon_degree=lon, south_lat_degree=lat, east_lon_degree=lon, north_lat_degree=lat),
        datum_name="WGS 84"
    )[0]
    utm_crs = CRS.from_epsg(utm_crs_info.code)

    # Projected coordinate system for accurate distance measurement
    proj_utm = Proj(utm_crs)

    # Convert the given lat/lon in WGS84 to UTM coordinates
    x, y = proj_utm(lon, lat)

    # Define the offsets (2km in each direction to form a 4km box)
    offset = 2000  # meters

    # Calculate the corners of the box in UTM coordinates
    bottom_left = (x - offset, y - offset)
    bottom_right = (x + offset, y - offset)
    top_right = (x + offset, y + offset)
    top_left = (x - offset, y + offset)

    # Create a polygon from these corners
    box = Polygon([bottom_left, bottom_right, top_right, top_left, bottom_left])

    # Optionally, convert the polygon back to geographic coordinates (WGS84)
    proj_wgs84 = Proj(proj='latlong', datum='WGS84')
    lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)

    # Create a new polygon in geographic coordinates
    geo_polygon = Polygon(zip(lon_lat_polygon[0], lon_lat_polygon[1]))

    return geo_polygon

meta['geometry'] = meta.apply(lambda row: get_4km_polygon_wkt(row['LOCATION_LAT'], row['LOCATION_LON']), axis=1)
meta

/tmp/ipykernel_74769/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_74769/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_74769/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_74769/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_7

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,TIME_INFO,geometry
0,AR-TF1,-54.973300,-66.733500,40.0,"{'ameriflux': ['2016_01_01', '2018_12_31']}",POLYGON ((-66.76372222733691 -54.9918372489149...
1,BR-CST,-7.968200,-38.384200,468.0,"{'ameriflux': ['2014_01_01', '2015_12_31']}",POLYGON ((-38.402320255670666 -7.9863166935648...
2,BR-Npw,-16.498000,-56.412000,120.0,"{'ameriflux': ['2013_01_01', '2017_12_31']}",POLYGON ((-56.430686316238486 -16.516130914339...
3,BR-Sa3,-3.018000,-54.971400,100.0,"{'fluxnet': ['2000_01_01', '2004_12_31']}",POLYGON ((-54.9893536894334 -3.036116479223139...
4,CA-ARB,52.695000,-83.945200,90.0,"{'ameriflux': ['2011_01_01', '2015_12_31']}",POLYGON ((-83.97353284847217 52.67630591740366...
...,...,...,...,...,...,...
381,CD-Ygb,0.814444,24.502472,466.0,"{'icos-2023': ['2020_01_01', '2022_12_31']}",POLYGON ((24.484525104157836 0.796355713851499...
382,IT-Niv,45.490910,7.139430,2750.0,"{'icos-2023': ['2019_01_01', '2022_12_31']}","POLYGON ((7.1144489857182 45.47249775232903, 7..."
383,FR-Mej,48.118400,-1.796350,40.0,"{'icos-2023': ['2019_01_01', '2022_12_31']}",POLYGON ((-1.823627784003099 48.10068804872159...
384,DK-Vng,56.037476,9.160710,67.7,"{'icos-2023': ['2020_01_01', '2022_12_31']}",POLYGON ((9.128551089546221 56.019544369789394...


In [9]:
gdf = gpd.GeoDataFrame(meta, crs='EPSG:4326', geometry=meta['geometry'])
gdf.to_file('sites.geojson', driver='GeoJSON')